# **Reto: Deserción de empleados**

Una empresa de productos médicos está interesada en atraer y mantener al mejor talento porque sabe que es la clave del éxito para cualquier organización. También toma en cuenta que, si un empleado abandona la empresa, se está provocando una pérdida de tiempo y dinero debido, entre otras cosas, a la inversión en capacitación y a la experiencia acumulada del empleado. Desde luego que hay algunas formas de deserción que son inevitables, como cuando un empleado se retira o cambia de ciudad de residencia. Sin embargo, existen algunos factores que se pueden controlar por parte de la empresa con el objetivo de minimizar la deserción al mejorar las condiciones de trabajo. A la empresa le interesa saber cuáles son los factores que hacen que un empleado siga con ellos y cuáles son los que se deben cambiar debido a que provocan que los empleados se vayan.

Aunque la empresa sabe que este estudio tiene muchas aristas desea empezar con algo muy simple: determinando si un empleado abandonará la empresa Attrition o no en un momento dado. Como se trata de un problema de clasificación binaria, lo más probable es que utilicen un modelo de machine learning. La empresa ha logrado recolectar 30 datos de 400 de sus empleados, pero no está segura si ese conjunto de datos sean los correctos para lo que pretende hacer, por lo que decidió contratarte como científico de datos para generar un set de datos adecuado para esta actividad.

Después de una revisión completa de los datos con los que se cuentan, has decidido escribir un programa en Pyhton Jupyter Notebook llamado RetoEmpleados.ipynb bajo el siguiente plan:

1. Importa las librerías requeridas.

In [1]:
# Importar las librerías requeridas
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

2. Lee el archivo CSV llamado empleadosRETO.csv y coloca los datos en un frame de Pandas llamado EmpleadosAttrition.

In [2]:
# Leer el archivo CSV y colocar los datos en un DataFrame de Pandas
EmpleadosAttrition = pd.read_csv('/content/empleadosRETO.csv')
EmpleadosAttrition.head()

,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,...,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsInCurrentRole,YearsSinceLastPromotion,Attrition
0,50,Travel_Rarely,Research & Development,1 km,2,Medical,1,997,4,Male,...,22,4,3,80,32,1,2,4,1,No
1,36,Travel_Rarely,Research & Development,6 km,2,Medical,1,178,2,Male,...,20,4,4,80,7,0,3,2,0,No
2,21,Travel_Rarely,Sales,7 km,1,Marketing,1,1780,2,Male,...,13,3,2,80,1,3,3,0,1,Yes
3,52,Travel_Rarely,Research & Development,7 km,4,Life Sciences,1,1118,2,Male,...,19,3,4,80,18,4,3,6,4,No
4,33,Travel_Rarely,Research & Development,15 km,1,Medical,1,582,2,Male,...,12,3,4,80,15,2,4,6,7,Yes


3. Elimina las columnas que, con alta probabilidad (estimada por ti), no tienen relación alguna con la salida. Hay algunas columnas que contienen información que no ayuda a definir el desgaste de un empleado, tal es caso de las siguientes:
  *   EmployeeCount: número de empleados, todos tienen un 1
  *   EmployeeNumber: ID del empleado, el cual es único para cada empleado
  *   Over18: mayores de edad, todos dicen “Y”
  *   StandardHours: horas de trabajo, todos tienen “80”

In [3]:
# Eliminar columnas que probablemente no tienen relación con la salida
columnas_a_eliminar = ['EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours']
EmpleadosAttrition.drop(columns=columnas_a_eliminar, inplace=True)
EmpleadosAttrition.head()

,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,JobInvolvement,JobLevel,...,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsInCurrentRole,YearsSinceLastPromotion,Attrition
0,50,Travel_Rarely,Research & Development,1 km,2,Medical,4,Male,3,4,...,No,22,4,3,32,1,2,4,1,No
1,36,Travel_Rarely,Research & Development,6 km,2,Medical,2,Male,3,2,...,No,20,4,4,7,0,3,2,0,No
2,21,Travel_Rarely,Sales,7 km,1,Marketing,2,Male,3,1,...,No,13,3,2,1,3,3,0,1,Yes
3,52,Travel_Rarely,Research & Development,7 km,4,Life Sciences,2,Male,3,3,...,No,19,3,4,18,4,3,6,4,No
4,33,Travel_Rarely,Research & Development,15 km,1,Medical,2,Male,3,3,...,Yes,12,3,4,15,2,4,6,7,Yes


4. Analizando la información proporcionada, detectaste que no se cuenta con los años que el empelado lleva en la compañía y parece ser un buen dato. Dicha cantidad se puede calcular con la fecha de contratación ‘HiringDate’:

  a.	Crea una columna llamada Year y obtén el año de contratación del empleado a partir de su fecha ‘HiringDate’. No se te olvide que debe ser un entero.

  b.	Crea una columna llamada YearsAtCompany que contenga los años que el empleado lleva en la compañía hasta el año 2018. Para su cálculo, usa la variable Year que acabas de crear.


In [4]:
# Convertir la columna 'HiringDate' a tipo datetime
try:
    EmpleadosAttrition['HiringDate'] = pd.to_datetime(EmpleadosAttrition['HiringDate'],format='%m/%d/%Y')
except ValueError as e:
    print("Error de conversión en las siguientes filas:")
    for indice, valor in EmpleadosAttrition['HiringDate'].items():
        try:
            pd.to_datetime(valor)
        except ValueError:
            print(f"Fila {indice}: {valor}")

Error de conversión en las siguientes filas:
Fila 229: 2/30/2012


In [5]:
# Corregir error
EmpleadosAttrition.at[229, 'HiringDate'] = '2/28/2012'

In [6]:
# Convertir la columna 'HiringDate' a tipo datetime
EmpleadosAttrition['HiringDate'] = pd.to_datetime(EmpleadosAttrition['HiringDate'],format='%m/%d/%Y')

# Crear una nueva columna 'Year' con el año de contratación del empleado
EmpleadosAttrition['Year'] = EmpleadosAttrition['HiringDate'].dt.year.astype(int)

# Calcular los años que el empleado lleva en la compañía hasta el año actual
año_actual = datetime.now().year
EmpleadosAttrition['YearsAtCompany'] = año_actual - EmpleadosAttrition['Year']

# Verificar los resultados
print(EmpleadosAttrition[['HiringDate', 'Year', 'YearsAtCompany']].head())

  HiringDate  Year  YearsAtCompany
0 2013-06-06  2013              11
1 2015-12-25  2015               9
2 2017-02-14  2017               7
3 2010-07-29  2010              14
4 2011-10-07  2011              13


5.	La DistanceFromHome está dada en kilómetros, pero tiene las letras “km” al final y así no puede ser entera:

  a.	Renombra la variable DistanceFromHome a DistanceFromHome_km.

  b.	Crea una nueva variable DistanceFromHome que sea entera, es decir, solo con números.

In [7]:
# Renombrar la variable DistanceFromHome a DistanceFromHome_km
EmpleadosAttrition.rename(columns={'DistanceFromHome': 'DistanceFromHome_km'}, inplace=True)

# Crear una nueva variable DistanceFromHome que sea entera
EmpleadosAttrition['DistanceFromHome'] = EmpleadosAttrition['DistanceFromHome_km'].str.replace(' km', '').astype(int)

# Verificar los resultados
print(EmpleadosAttrition[['DistanceFromHome']].head())

   DistanceFromHome
0                 1
1                 6
2                 7
3                 7
4                15


6.	Borra las columnas Year, HiringDate y DistanceFromHome_km debido a que ya no son útiles.

In [8]:
# Eliminar las columnas 'Year', 'HiringDate' y 'DistanceFromHome_km'
EmpleadosAttrition.drop(columns=['Year', 'HiringDate', 'DistanceFromHome_km'], inplace=True)

7.	Aprovechando los ajustes que se están haciendo, la empresa desea saber si todos los departamentos tienen un ingreso promedio similar. Genera una nuevo frame llamado SueldoPromedioDepto que contenga el MonthlyIncome promedio por departamento de los empleados y colócalo en una variable llamada SueldoPromedio. Esta tabla solo es informativa, no la vas a utilizar en el set de datos que estás construyendo.

In [9]:
# Calcular el ingreso promedio por departamento
SueldoPromedioDepto = EmpleadosAttrition.groupby('Department')['MonthlyIncome'].mean().reset_index()

# Renombrar la columna MonthlyIncome a SueldoPromedio
SueldoPromedioDepto.rename(columns={'MonthlyIncome': 'SueldoPromedio'}, inplace=True)

# Visualizar el DataFrame SueldoPromedioDepto
SueldoPromedioDepto.head()

,Department,SueldoPromedio
0,Human Resources,6239.888889
1,Research & Development,6804.149813
2,Sales,7188.250000


8.	La variable MonthlyIncome tiene un valor numérico muy grande comparada con las otras variables. Escala dicha variable para que tenga un valor entre 0 y 1.

In [10]:
# Inicializar el escalador
scaler = MinMaxScaler()

# Escalar la variable MonthlyIncome
EmpleadosAttrition['MonthlyIncome_MinMaxSscaled'] = scaler.fit_transform(EmpleadosAttrition[['MonthlyIncome']])
EmpleadosAttrition.head()

,Age,BusinessTravel,Department,Education,EducationField,EnvironmentSatisfaction,Gender,JobInvolvement,JobLevel,JobRole,...,RelationshipSatisfaction,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsInCurrentRole,YearsSinceLastPromotion,Attrition,YearsAtCompany,DistanceFromHome,MonthlyIncome_MinMaxSscaled
0,50,Travel_Rarely,Research & Development,2,Medical,4,Male,3,4,Research Director,...,3,32,1,2,4,1,No,11,1,0.864269
1,36,Travel_Rarely,Research & Development,2,Medical,2,Male,3,2,Manufacturing Director,...,4,7,0,3,2,0,No,9,6,0.207340
2,21,Travel_Rarely,Sales,1,Marketing,2,Male,3,1,Sales Representative,...,2,1,3,3,0,1,Yes,7,7,0.088062
3,52,Travel_Rarely,Research & Development,4,Life Sciences,2,Male,3,3,Healthcare Representative,...,4,18,4,3,6,4,No,14,7,0.497574
4,33,Travel_Rarely,Research & Development,1,Medical,2,Male,3,3,Manager,...,4,15,2,4,6,7,Yes,13,15,0.664470


9.	Todo parece indicar que las variables categóricas que quedan sí son importantes para obtener la variable de salida. Convierte todas las variables categóricas que quedan a numéricas:

  a.	BusinessTravel

  b.	Department

  c.	EducationField

  d.	Gender

  e.	JobRole

  f.	MaritalStatus

  g.	Attrition

In [11]:
# Obtener los valores únicos de las variables categóricas
categorias = ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'Attrition']

for categoria in categorias:
    valores_unicos = EmpleadosAttrition[categoria].unique()
    print(f"Valores únicos para la variable '{categoria}': {valores_unicos}")

Valores únicos para la variable 'BusinessTravel': ['Travel_Rarely' 'Non-Travel' 'Travel_Frequently' nan]
Valores únicos para la variable 'Department': ['Research & Development' 'Sales' 'Human Resources']
Valores únicos para la variable 'EducationField': ['Medical' 'Marketing' 'Life Sciences' 'Technical Degree' 'Other'
 'Human Resources']
Valores únicos para la variable 'Gender': ['Male' 'Female']
Valores únicos para la variable 'JobRole': ['Research Director' 'Manufacturing Director' 'Sales Representative'
 'Healthcare Representative' 'Manager' 'Sales Executive'
 'Laboratory Technician' 'Research Scientist' 'Human Resources']
Valores únicos para la variable 'MaritalStatus': ['Divorced' 'Single' 'Married' nan]
Valores únicos para la variable 'Attrition': ['No' 'Yes']


In [12]:
# Menos de 10 categorías unicas para todas las variables
# Convertir variables categóricas a numéricas usando one-hot encoding
EmpleadosAttrition = pd.get_dummies(EmpleadosAttrition, columns=categorias, drop_first=True)

In [13]:
EmpleadosAttrition.columns

Index(['Age', 'Education', 'EnvironmentSatisfaction', 'JobInvolvement',
       'JobLevel', 'JobSatisfaction', 'MonthlyIncome', 'NumCompaniesWorked',
       'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'TotalWorkingYears',
       'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsInCurrentRole',
       'YearsSinceLastPromotion', 'YearsAtCompany', 'DistanceFromHome',
       'MonthlyIncome_MinMaxSscaled', 'BusinessTravel_Travel_Frequently',
       'BusinessTravel_Travel_Rarely', 'Department_Research & Development',
       'Department_Sales', 'EducationField_Life Sciences',
       'EducationField_Marketing', 'EducationField_Medical',
       'EducationField_Other', 'EducationField_Technical Degree',
       'Gender_Male', 'JobRole_Human Resources',
       'JobRole_Laboratory Technician', 'JobRole_Manager',
       'JobRole_Manufacturing Director', 'JobRole_Research Director',
       'JobRole_Research Scientist', 'JobRole_Sales Executive',
       'Job

In [14]:
EmpleadosAttrition.head()

,Age,Education,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,NumCompaniesWorked,OverTime,PercentSalaryHike,...,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Married,MaritalStatus_Single,Attrition_Yes
0,50,2,4,3,4,4,17399,9,No,22,...,0,0,0,1,0,0,0,0,0,0
1,36,2,2,3,2,2,4941,6,No,20,...,0,0,1,0,0,0,0,0,0,0
2,21,1,2,3,1,2,2679,1,No,13,...,0,0,0,0,0,0,1,0,1,1
3,52,4,2,3,3,2,10445,7,No,19,...,0,0,0,0,0,0,0,0,1,0
4,33,1,2,3,3,3,13610,7,Yes,12,...,0,1,0,0,0,0,0,1,0,1


10.	Ahora debes hacer la evaluación de las variables para quedarte con las mejores. Calcula la correlación lineal de cada una de las variables con respecto al Attrition.

In [15]:
# Calcular la correlación lineal de cada variable con respecto a 'Attrition'
correlaciones = EmpleadosAttrition.corr()['Attrition_Yes'].sort_values(ascending=False)

# Visualizar las correlaciones
print(correlaciones)

Attrition_Yes                        1.000000
MaritalStatus_Single                 0.205849
JobRole_Sales Representative         0.191294
EducationField_Technical Degree      0.129104
JobRole_Laboratory Technician        0.125264
Department_Sales                     0.066116
DistanceFromHome                     0.052732
BusinessTravel_Travel_Rarely         0.042755
BusinessTravel_Travel_Frequently     0.035387
JobRole_Human Resources              0.032714
EducationField_Marketing             0.016768
JobRole_Research Scientist           0.007977
JobRole_Sales Executive             -0.003115
EducationField_Other                -0.004275
PerformanceRating                   -0.006471
NumCompaniesWorked                  -0.009082
WorkLifeBalance                     -0.021723
EducationField_Life Sciences        -0.027457
Gender_Male                         -0.028839
RelationshipSatisfaction            -0.030945
JobRole_Manufacturing Director      -0.042404
EducationField_Medical            

<ipython-input-15-d9c2dd305f6e>:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlaciones = EmpleadosAttrition.corr()['Attrition_Yes'].sort_values(ascending=False)


11.	Selecciona solo aquellas variables que tengan una correlación mayor o igual a 0.1, dejándolas en otro frame llamado EmpleadosAttritionFinal. No olvides mantener la variable de salida Attrition; esto es equivalente a borrar las que no cumplen con el límite.

In [16]:
# Definir la correlación mínima deseada
correlacion_minima = 0.1

# Calcular la correlación de cada variable con respecto a 'Attrition_Yes'
correlaciones_absolutas = EmpleadosAttrition.corr()['Attrition_Yes'].abs()

# Seleccionar las variables con correlación mayor o igual a 0.1 con 'Attrition_Yes'
variables_seleccionadas = correlaciones_absolutas[correlaciones_absolutas >= correlacion_minima].index

# Crear un nuevo DataFrame con las variables seleccionadas y la variable de salida 'Attrition_Yes'
EmpleadosAttritionFinal = EmpleadosAttrition[variables_seleccionadas]
EmpleadosAttritionFinal['Attrition_Yes'] = EmpleadosAttrition['Attrition_Yes']

<ipython-input-16-7ac29f6bc102>:5: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlaciones_absolutas = EmpleadosAttrition.corr()['Attrition_Yes'].abs()
<ipython-input-16-7ac29f6bc102>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  EmpleadosAttritionFinal['Attrition_Yes'] = EmpleadosAttrition['Attrition_Yes']


In [17]:
EmpleadosAttritionFinal.columns

Index(['Age', 'EnvironmentSatisfaction', 'JobInvolvement', 'JobLevel',
       'JobSatisfaction', 'MonthlyIncome', 'TotalWorkingYears',
       'YearsInCurrentRole', 'YearsAtCompany', 'MonthlyIncome_MinMaxSscaled',
       'EducationField_Technical Degree', 'JobRole_Laboratory Technician',
       'JobRole_Research Director', 'JobRole_Sales Representative',
       'MaritalStatus_Single', 'Attrition_Yes'],
      dtype='object')

In [18]:
EmpleadosAttritionFinal.head()

,Age,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,TotalWorkingYears,YearsInCurrentRole,YearsAtCompany,MonthlyIncome_MinMaxSscaled,EducationField_Technical Degree,JobRole_Laboratory Technician,JobRole_Research Director,JobRole_Sales Representative,MaritalStatus_Single,Attrition_Yes
0,50,4,3,4,4,17399,32,4,11,0.864269,0,0,1,0,0,0
1,36,2,3,2,2,4941,7,2,9,0.207340,0,0,0,0,0,0
2,21,2,3,1,2,2679,1,0,7,0.088062,0,0,0,1,1,1
3,52,2,3,3,2,10445,18,6,14,0.497574,0,0,0,0,1,0
4,33,2,3,3,3,13610,15,6,13,0.664470,0,0,0,0,0,1


12.	Crea una nueva variable llamada EmpleadosAttritionPCA formada por los componentes principales del frame EmpleadosAttritionFinal. Recuerda que el resultado del proceso PCA es un numpy array, por lo que, para hacer referencia a una columna, por ejemplo, la 0, puedes usar la instrucción EmpleadosAttritionPCA[:,0]).

In [19]:
# Inicializar PCA
pca = PCA()

# Aplicar PCA al DataFrame EmpleadosAttritionFinal
componentes_principales = pca.fit_transform(EmpleadosAttritionFinal.drop(columns=['Attrition_Yes']))

# Crear DataFrame para EmpleadosAttritionPCA con los componentes principales
EmpleadosAttritionPCA = pd.DataFrame(componentes_principales)

In [20]:
# Inicializar el objeto PCA con el número de componentes deseados
pca = PCA()

# Aplicar PCA al DataFrame EmpleadosAttritionFinal
EmpleadosAttritionPCA = pca.fit_transform(EmpleadosAttritionFinal.drop(columns=['Attrition_Yes']))

# Crear un nuevo DataFrame para EmpleadosAttritionPCA
EmpleadosAttritionPCA = pd.DataFrame(EmpleadosAttritionPCA)

In [21]:
EmpleadosAttritionPCA.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,10488.493617,5.863734,-6.595284,8.619339,-1.886956,-0.336530,-2.046633,-0.488484,-0.321753,0.434121,-0.454869,0.227326,0.423172,-0.054388,1.241056e-15
1,-1969.533472,-1.968460,-4.113971,-0.612987,0.193211,0.143550,0.927173,-0.308904,-0.260355,-0.401339,0.157321,-0.109386,0.052770,-0.028659,-1.383897e-16
2,-4231.551535,-15.390077,-3.245180,2.032129,0.963035,0.110689,0.901487,-0.317087,0.697859,-0.112702,-0.369988,0.319164,-0.050273,0.549738,5.030336e-17
3,3534.489566,10.656540,-2.906493,-2.279980,-1.002745,0.230295,0.958348,-0.211129,0.811775,0.030817,0.061417,-0.115081,-0.196035,-0.016315,-6.698157e-16
4,6699.466534,-11.780415,-2.749785,1.183256,-1.773981,-0.557500,0.273126,-0.446783,-0.350266,0.256920,-0.361017,-0.260855,-0.398406,0.036753,-9.708198e-17


13.	Agrega el mínimo número de Componentes Principales en columnas del frame EmpleadosAttritionPCA que logren explicar el 80% de la varianza, al frame EmpleadosAttritionFinal. Puedes usar la instrucción assign, columna por columna, llamando a cada una C0, C1, etc., hasta las que vayas a agregar.

In [22]:
# Agregar el mínimo número de Componentes Principales que explican el 80% de la varianza
varianza_explicada = pca.explained_variance_ratio_
varianza_acumulada = np.cumsum(varianza_explicada)
num_componentes_minimos = np.argmax(varianza_acumulada >= 0.8) + 1

# Agregar las nuevas columnas al DataFrame EmpleadosAttritionFinal
for i in range(num_componentes_minimos):
    EmpleadosAttritionFinal[f'C{i+1}'] = EmpleadosAttritionPCA.iloc[:, i]

EmpleadosAttritionFinal.columns

<ipython-input-22-bc093d24495b>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  EmpleadosAttritionFinal[f'C{i+1}'] = EmpleadosAttritionPCA.iloc[:, i]


Index(['Age', 'EnvironmentSatisfaction', 'JobInvolvement', 'JobLevel',
       'JobSatisfaction', 'MonthlyIncome', 'TotalWorkingYears',
       'YearsInCurrentRole', 'YearsAtCompany', 'MonthlyIncome_MinMaxSscaled',
       'EducationField_Technical Degree', 'JobRole_Laboratory Technician',
       'JobRole_Research Director', 'JobRole_Sales Representative',
       'MaritalStatus_Single', 'Attrition_Yes', 'C1'],
      dtype='object')

In [23]:
EmpleadosAttritionFinal.head()

,Age,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,TotalWorkingYears,YearsInCurrentRole,YearsAtCompany,MonthlyIncome_MinMaxSscaled,EducationField_Technical Degree,JobRole_Laboratory Technician,JobRole_Research Director,JobRole_Sales Representative,MaritalStatus_Single,Attrition_Yes,C1
0,50,4,3,4,4,17399,32,4,11,0.864269,0,0,1,0,0,0,10488.493617
1,36,2,3,2,2,4941,7,2,9,0.207340,0,0,0,0,0,0,-1969.533472
2,21,2,3,1,2,2679,1,0,7,0.088062,0,0,0,1,1,1,-4231.551535
3,52,2,3,3,2,10445,18,6,14,0.497574,0,0,0,0,1,0,3534.489566
4,33,2,3,3,3,13610,15,6,13,0.664470,0,0,0,0,0,1,6699.466534


14.	Guarda el set de datos que has formado y que tienes en EmpleadosAttritionFinal en un archivo CSV llamado EmpleadosAttritionFinal.csv. Las últimas columnas que colocaste quedarán después de la variable Attrition, lo cual no importa, pero si gustas lo puedes arreglar antes de escribir el archivo.

In [24]:
# Reorganizar las columnas para que 'Attrition_Yes' quede al final
columnas_ordenadas = [col for col in EmpleadosAttritionFinal.columns if col != 'Attrition_Yes'] + ['Attrition_Yes']
EmpleadosAttritionFinal = EmpleadosAttritionFinal[columnas_ordenadas]

EmpleadosAttritionFinal.columns

Index(['Age', 'EnvironmentSatisfaction', 'JobInvolvement', 'JobLevel',
       'JobSatisfaction', 'MonthlyIncome', 'TotalWorkingYears',
       'YearsInCurrentRole', 'YearsAtCompany', 'MonthlyIncome_MinMaxSscaled',
       'EducationField_Technical Degree', 'JobRole_Laboratory Technician',
       'JobRole_Research Director', 'JobRole_Sales Representative',
       'MaritalStatus_Single', 'C1', 'Attrition_Yes'],
      dtype='object')

In [25]:
EmpleadosAttritionFinal.head()

,Age,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,TotalWorkingYears,YearsInCurrentRole,YearsAtCompany,MonthlyIncome_MinMaxSscaled,EducationField_Technical Degree,JobRole_Laboratory Technician,JobRole_Research Director,JobRole_Sales Representative,MaritalStatus_Single,C1,Attrition_Yes
0,50,4,3,4,4,17399,32,4,11,0.864269,0,0,1,0,0,10488.493617,0
1,36,2,3,2,2,4941,7,2,9,0.207340,0,0,0,0,0,-1969.533472,0
2,21,2,3,1,2,2679,1,0,7,0.088062,0,0,0,1,1,-4231.551535,1
3,52,2,3,3,2,10445,18,6,14,0.497574,0,0,0,0,1,3534.489566,0
4,33,2,3,3,3,13610,15,6,13,0.664470,0,0,0,0,0,6699.466534,1


In [26]:
# Guardar el DataFrame en un archivo CSV
EmpleadosAttritionFinal.to_csv('/content/EmpleadosAttritionFinal.csv', index=False)